# Mypy

## Introducción 
[Mypy](https://mypy.readthedocs.io/en/stable/index.html) es un verificador de tipo estático opcional para Python que tiene como objetivo combinar los beneficios de la escritura dinámica y la escritura estática. 

Mypy combina el poder expresivo y la conveniencia de Python con un poderoso sistema de tipos y verificación de tipos en tiempo de compilación. El tipo Mypy comprueba los programas estándar de Python; ejecútelos usando cualquier máquina virtual de Python sin básicamente una sobrecarga de tiempo de ejecución.

### Escritura dinámica y estática
<img src="images/html_01.png"  align = "center">

Migre el código existente a escritura estática, una función a la vez. Puede mezclar libremente la escritura estática y dinámica dentro de un programa, dentro de un módulo o dentro de una expresión. No es necesario renunciar a la escritura dinámica: utilice la escritura estática cuando tenga sentido. A menudo, el simple hecho de agregar firmas de funciones le proporciona un código escrito estáticamente. Mypy puede inferir los tipos de otras variables.

### Sintaxis de Python
El tipo Mypy comprueba los programas que tienen anotaciones de tipo que cumplen con [PEP 484](https://www.python.org/dev/peps/pep-0484/). Empezar es fácil si conoce Python. El objetivo es admitir casi todas las construcciones del lenguaje Python en mypy.

### Instalación

Mypy requiere Python 3.5 o una versión superior para ejecutarse. Para instalar Mypy se debe ejecutar por terminal:

```terminal
# using pip (pip3)
$ pip install mypy

# using pipenv
$ pipenv install mypy

# using poetry
$ poetry add mypy
```

### Uso

Para archivos de manera individual (ejemplo `algo.py`), se debe ejecutar por terminal la sentencia:
```terminal
$ mypy algo.py
```



## Formas de uso

Aquí hay algunos programas de ejemplo de mypy. Cada ejemplo tiene código Python/mypy escrito dinámicamente y código mypy escrito estáticamente equivalente lado a lado. Todos los programas siguen siendo válidos en Python 3.x. Se destacan todas las diferencias entre las variantes.

### Frecuencias de palabras con un diccionario

En este ejemplo, agregamos una declaración de tipo explícita para la variable `d`, ya que no es obvio en el contexto local.

<img src="images/html_02.png"   align = "center">


#### Mypy with static typing

In [1]:
%%writefile example_01.py
# Display the frequencies of words in a file.
import sys
import re


if not sys.argv[1:]:
    raise RuntimeError('Usage: wordfreq FILE')

d = {}

with open(sys.argv[1]) as f:
    for s in f:
        for word in re.sub('\W', ' ', s).split():
            d[word] = d.get(word, 0) + 1

# Use list comprehension
l = [(freq, word) for word, freq in d.items()]

for freq, word in sorted(l):
    print('%-6d %s' % (freq, word))

Overwriting example_01.py


In [2]:
!mypy ./example_01.py

example_01.py:9: error: Need type annotation for 'd' (hint: "d: Dict[<type>, <type>] = ...")
Found 1 error in 1 file (checked 1 source file)


#### Mypy with dynamic typing

In [3]:
%%writefile example_02.py
# Display the frequencies of words in a file.

import sys
import re
from typing import Dict

if not sys.argv[1:]:
    raise RuntimeError('Usage: wordfreq FILE')

d = {}  # type: Dict[str, int]

with open(sys.argv[1]) as f:
    for s in f:
        for word in re.sub('\W', ' ', s).split():
            d[word] = d.get(word, 0) + 1

# Use list comprehension
l = [(freq, word) for word, freq in d.items()]

for freq, word in sorted(l):
    print('%-6d %s' % (freq, word))

Writing example_02.py


In [4]:
!mypy ./example_02.py

Success: no issues found in 1 source file


### Clase simple

En este ejemplo, elegimos usar números enteros para representar el balance. Esto estaría bien en un juego, por ejemplo, pero en otras aplicaciones, un tipo diferente tendría más sentido.

<img src="images/html_03.png"   align = "center">


#### Mypy with static typing

In [5]:
%%writefile example_01.py
class BankAccount:
    def __init__(self, initial_balance=0):
        self.balance = initial_balance
    def deposit(self, amount):
        self.balance += amount
    def withdraw(self, amount):
        self.balance -= amount
    def overdrawn(self):
        return self.balance < 0

my_account = BankAccount(15)
my_account.withdraw(5)
print(my_account.balance)

Overwriting example_01.py


In [6]:
!mypy example_01.py

Success: no issues found in 1 source file


#### Mypy with dynamic typing

In [7]:
%%writefile example_02.py
class BankAccount:
    def __init__(self, initial_balance: int = 0) -> None:
        self.balance = initial_balance
    def deposit(self, amount: int) -> None:
        self.balance += amount
    def withdraw(self, amount: int) -> None:
        self.balance -= amount
    def overdrawn(self) -> bool:
        return self.balance < 0

my_account = BankAccount(15)
my_account.withdraw(5)
print(my_account.balance)

Overwriting example_02.py


In [8]:
!mypy ./example_02.py

Success: no issues found in 1 source file


### Números primos con generadores

Como el ejemplo de la cuenta bancaria, esto fue adaptado de la wiki de Python.

<img src="images/html_04.png"  align = "center">


#### Mypy with static typing

In [9]:
%%writefile example_01.py
import itertools


def iter_primes():
    # An iterator of all numbers between 2 and
    # +infinity
    numbers = itertools.count(2)

    # Generate primes forever
    while True:
        # Get the first number from the iterator
        # (always a prime)
        prime = next(numbers)
        yield prime

        # This code iteratively builds up a chain
        # of filters...
        numbers = filter(prime.__rmod__, numbers)

for p in iter_primes():
    if p > 1000:
        break
    print(p)

Overwriting example_01.py


In [10]:
!mypy ./example_01.py

Success: no issues found in 1 source file


#### Mypy with dynamic typing

In [11]:
%%writefile example_02.py
import itertools
from typing import Iterator

def iter_primes() -> Iterator[int]:
    # An iterator of all numbers between 2 and
    # +infinity
    numbers = itertools.count(2)

    # Generate primes forever
    while True:
        # Get the first number from the iterator
        # (always a prime)
        prime = next(numbers)
        yield prime

        # This code iteratively builds up a chain
        # of filters...
        numbers = filter(prime.__rmod__, numbers)

for p in iter_primes():
    if p > 1000:
        break
    print(p)

Overwriting example_02.py


In [12]:
!mypy ./example_02.py

Success: no issues found in 1 source file


**Observación**: al final de cada presentación, se eliminan los archivos que generamos de manera temporal.

In [13]:
!rm -r example_01.py example_02.py